In [1]:
import psycopg2
import pandas as pd
import csv

In [2]:
config = {
    "user": "postgres",
    "password": "admin",
    "database": "etl_db"
}

def create_connection():
    try:
        cnx = psycopg2.connect(
            host='localhost',
            user=config["user"],
            password=config["password"],
            database=config["database"]
        )
        print('Conexión exitosa!!')
    except psycopg2.Error as e:
        cnx = None
        print('No se puede conectar:', e)
    return cnx

In [3]:
create_connection()

Conexión exitosa!!


<connection object at 0x0000013A3C08FE10; dsn: 'user=postgres password=xxx dbname=etl_db host=localhost', closed: 0>

In [4]:
data = pd.read_csv('candidates (1).csv', sep=";")
data

,First Name,Last Name,Email,Application Date,Country,YOE,Seniority,Technology,Code Challenge Score,Technical Interview Score
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7
...,...,...,...,...,...,...,...,...,...,...
49995,Bethany,Shields,rocky_mitchell@hotmail.com,2022-01-09,Dominican Republic,27,Trainee,Security,2,1
49996,Era,Swaniawski,dolores.roob@hotmail.com,2020-06-02,Morocco,21,Lead,Game Development,1,2
49997,Martin,Lakin,savanah.stracke@gmail.com,2018-12-15,Uganda,20,Trainee,System Administration,6,1
49998,Aliya,Abernathy,vivienne.fritsch@yahoo.com,2020-05-30,Czech Republic,20,Senior,Database Administration,0,0


In [5]:
creacion = """
CREATE TABLE Applicants (
    id SERIAL PRIMARY KEY,
    first_name VARCHAR(100),
    last_name VARCHAR(100),
    email VARCHAR(100),
    application_date DATE,
    country VARCHAR(250),
    yoe INTEGER,
    seniority VARCHAR(250),
    technology VARCHAR(100),
    code_challenge_score FLOAT,
    technical_interview_score FLOAT
);
"""

try:
    conexion = create_connection()
    cursor = conexion.cursor()
    
    cursor.execute(creacion)
    conexion.commit()
    print("Tabla creada exitosamente.")
    
except psycopg2.Error as e:
    print("Error al crear la tabla:", e)
    
finally:
    if conexion:
        conexion.close()


Conexión exitosa!!
Tabla creada exitosamente.


In [ ]:
cnx = None
insert_query = """
INSERT INTO Applicants (first_name, last_name, email, application_date, country, yoe, seniority,technology, code_challenge_score, technical_interview_score)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
try:
    cnx = create_connection()
    cur = cnx.cursor()
    for index, row in data.iterrows():
        values = (
            row['First Name'], row['Last Name'], row['Email'], row['Application Date'], row['Country'],
            row['YOE'], row['Seniority'], row['Technology'], row['Code Challenge Score'], row['Technical Interview Score']
        )
        #print(values)
        cur.execute(insert_query, values)
    cur.close()
    cnx.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if cnx is not None:
        cnx.close()

Conexión exitosa!!
('Bernadette', 'Langworth', 'leonard91@yahoo.com', '2021-02-26', 'Norway', 2, 'Intern', 'Data Engineer', 3, 3)
('Camryn', 'Reynolds', 'zelda56@hotmail.com', '2021-09-09', 'Panama', 10, 'Intern', 'Data Engineer', 2, 10)
('Larue', 'Spinka', 'okey_schultz41@gmail.com', '2020-04-14', 'Belarus', 4, 'Mid-Level', 'Client Success', 10, 9)
('Arch', 'Spinka', 'elvera_kulas@yahoo.com', '2020-10-01', 'Eritrea', 25, 'Trainee', 'QA Manual', 7, 1)
('Larue', 'Altenwerth', 'minnie.gislason@gmail.com', '2020-05-20', 'Myanmar', 13, 'Mid-Level', 'Social Media Community Management', 9, 7)
('Alec', 'Abbott', 'juanita_hansen@gmail.com', '2019-08-17', 'Zimbabwe', 8, 'Junior', 'Adobe Experience Manager', 2, 9)
('Allison', 'Jacobs', 'alba_rolfson27@yahoo.com', '2018-05-18', 'Wallis and Futuna', 19, 'Trainee', 'Sales', 2, 9)
('Nya', 'Skiles', 'madisen.zulauf@gmail.com', '2021-12-09', 'Myanmar', 1, 'Lead', 'Mulesoft', 2, 5)
('Mose', 'Lakin', 'dale_murazik@hotmail.com', '2018-03-13', 'Italy', 18

In [9]:



create_table_query = """
CREATE TABLE ContractedApplicants (
    id SERIAL PRIMARY KEY,
    first_name VARCHAR(100),
    last_name VARCHAR(100),
    email VARCHAR(100),
    application_date DATE,
    country VARCHAR(250),
    yoe INTEGER,
    seniority VARCHAR(250),
    technology VARCHAR(100),
    code_challenge_score FLOAT,
    technical_interview_score FLOAT,
    is_hired BOOLEAN
);
"""


insert_query = """
INSERT INTO ContractedApplicants (first_name, last_name, email, application_date, country, yoe, seniority, technology, code_challenge_score, technical_interview_score, is_hired)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

cnx = None
try:
    cnx = create_connection()
    cur = cnx.cursor()
    cur.execute(create_table_query)
    cur.close()
    cnx.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if cnx is not None:
        cnx.close()

cnx = None
try:
    cnx = create_connection()
    cur = cnx.cursor()
    
    for index, row in data.iterrows():
        code_challenge_score = row['Code Challenge Score']
        technical_interview_score = row['Technical Interview Score']
        is_hired = code_challenge_score >= 7 and technical_interview_score >= 7
        
        values = (
            row['First Name'], row['Last Name'], row['Email'], row['Application Date'], row['Country'],
            row['YOE'], row['Seniority'], row['Technology'], code_challenge_score, technical_interview_score, is_hired
        )
        cur.execute(insert_query, values)
    
    cur.close()
    cnx.commit()
    print("Datos insertados exitosamente en la tabla ContractedApplicants.")
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if cnx is not None:
        cnx.close()



Conexión exitosa!!
Conexión exitosa!!
Datos insertados exitosamente en la tabla ContractedApplicants.


In [14]:
!pip install matplotlib


  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/6d/f8/ff4acac6ea3f896146fd2a9f76dafb7c36973f2a329cae1d60a7c7252395/matplotlib-3.7.2-cp38-cp38-win_amd64.whl.metadata
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/b2/e5/6a7a6f2bdfcc0a235adf6f40be4f0ab5d23e65b766af1b2570c26b33d3b3/contourpy-1.1.0-cp38-cp38-win_amd64.whl.metadata
  Obtaining dependency information for fonttools>=4.22.0 from https://files.pythonhosted.org/packages/ee/d1/405b6d7a84cfd43cad518bf3d243433d637ada0add65e93110f5f480f86a/fonttools-4.42.1-cp38-cp38-win_amd64.whl.metadata
     ---------------------------------------- 0.0/154.1 kB ? eta -:--:--
     ------------------ -------------------- 71.7/154.1 kB 1.3 MB/s eta 0:00:01
     -------------------------------------  153.6/154.1 kB 1.8 MB/s eta 0:00:01
     -------------------------------------- 154.1/154.1 kB 1.2 MB/s eta 0:00:00
  Obtaining dependency information for 